In [1]:
import pandas as pd


In [4]:
df=pd.read_csv('jobs_dataset_with_features.csv')

In [ ]:
df.shape

(1615940, 2)

In [6]:
df.head()

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."
2,Quality Control Manager,0 to 12 Years Operations Manager PhD Quality c...
3,Wireless Network Engineer,4 to 11 Years Network Engineer PhD Wireless ne...
4,Conference Manager,1 to 12 Years Event Manager MBA Event planning...


In [8]:
df['Role'].value_counts()

Role
Interaction Designer            20580
Network Administrator           17470
User Interface Designer         14036
Social Media Manager            13945
User Experience Designer        13935
                                ...  
Inventory Control Specialist     3342
Budget Analyst                   3335
Clinical Nurse Manager           3324
Social Science Researcher        3321
Paid Advertising Specialist      3306
Name: count, Length: 376, dtype: int64

In [9]:
len(df['Role'].value_counts())

376

Dropping classes with less than 6500 instances


In [10]:
min_count=6500
role_counts=df['Role'].value_counts()
dropped_classes=role_counts[role_counts<min_count].index
filtered_df=df[~df['Role'].isin(dropped_classes)].reset_index(drop=True)

filtered_df['Role'].value_counts()

Role
Interaction Designer          20580
Network Administrator         17470
User Interface Designer       14036
Social Media Manager          13945
User Experience Designer      13935
                              ...  
Benefits Coordinator           6839
Research Analyst               6830
Administrative Coordinator     6803
IT Support Specialist          6799
UI/UX Designer                 6743
Name: count, Length: 61, dtype: int64

In [12]:
len(filtered_df['Role'].value_counts())

61

In [13]:
df=filtered_df.sample(n=100000)

In [14]:
df.head()

,Role,Features
445132,Backend Developer,4 to 10 Years Software Engineer B.Com Proficie...
136669,Network Administrator,1 to 12 Years Network Engineer B.Tech Network ...
356895,Quality Assurance Analyst,0 to 12 Years Software Tester B.Com Quality as...
424299,Interaction Designer,2 to 9 Years UX/UI Designer BA Interaction des...
67055,Event Planner,1 to 15 Years Event Coordinator B.Tech Event p...


TFIDF

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

X=df['Features']
y=df['Role']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

tfidf_vectorizer=TfidfVectorizer()
X_train_tfidf=tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf=tfidf_vectorizer.transform(X_test)

In [16]:
rf_classifier=RandomForestClassifier()
rf_classifier.fit(X_train_tfidf,y_train)

y_pred=rf_classifier.predict(X_test_tfidf)
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy",accuracy)

Accuracy 1.0


Recommendation System

In [17]:
import re
def cleanResume(txt):
    cleanText=re.sub('http\S+\s',' ',txt)
    cleanText=re.sub('RT|cc',' ',cleanText)
    cleanText=re.sub('#\S+\s',' ',cleanText)
    cleanText=re.sub('@\S+',' ',cleanText)
    cleanText=re.sub('[%s]'% re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{\}~"""),' ',cleanText)
    cleanText=re.sub(r'[^\x00-\x7f]',' ',cleanText)
    cleanText=re.sub('\s+',' ',cleanText)
    return cleanText.strip()


In [18]:
def job_recommendation(resume_text):
    resume_text=cleanResume(resume_text)
    resume_tfidf=tfidf_vectorizer.transform([resume_text])
    predicted_category=rf_classifier.predict(resume_tfidf)[0]
    return predicted_category

In [19]:
resume_file="hr assistant hard work dedicated detailed oriented experienced"
predicted_category=job_recommendation(resume_file)
print("Predicted Category:",predicted_category)

Predicted Category: Benefits Coordinator


In [20]:
resume_file = """
Experienced IT Professional with 5+ years in managing hardware, software, and network systems in enterprise environments. 
Proven ability to provide technical support, maintain systems uptime, and implement security policies. Adept in system administration, 
troubleshooting, and customer service.

Work Experience:
- IT Support Specialist at TechZone Ltd. (2019–Present)
  - Provided Tier 1 and Tier 2 support for over 300 users across multiple departments
  - Managed Active Directory, user accounts, and group policies
  - Installed, configured, and maintained Windows servers, desktops, and printers
  - Troubleshot network connectivity and VPN issues
  - Maintained asset inventory and procurement of hardware/software

- System Administrator Intern at InfoGrid Solutions (2018–2019)
  - Assisted in server patching, OS upgrades, and firewall configuration
  - Monitored network activity using Wireshark and SolarWinds

Education:
- B.Sc. in Information Technology, ABC Institute of Tech (2015–2018)

Certifications:
- CompTIA A+
- Microsoft Certified: Azure Fundamentals
- Cisco CCNA (in progress)

Skills:
- Operating Systems: Windows, Linux (Ubuntu/CentOS)
- Networking: DNS, DHCP, VPN, TCP/IP
- Tools: Active Directory, SCCM, Jira, VMware
- Soft Skills: Communication, problem-solving, time management
"""


predicted_category=job_recommendation(resume_file)
print("Predicted Category:",predicted_category)

Predicted Category: Network Administrator


In [21]:
resume_file = """
Creative professional with over 4 years of experience crafting engaging visuals for digital and print platforms. 
Specialized in layout design, branding, and user interface design. Adept at translating ideas into compelling graphics 
using industry-standard tools.

Work Experience:
- Visual Design Specialist at PixelWorks Agency (2020–Present)
  - Developed marketing assets including brochures, posters, web banners, and social media content
  - Designed logos and brand kits for startups and established brands
  - Collaborated with marketing teams to ensure brand consistency across channels
  - Built interactive UI mockups for web and mobile apps using Figma and Adobe XD

- Design Associate at CreativeEdge Studio (2018–2020)
  - Assisted senior team in image editing, layout structuring, and visual hierarchy optimization
  - Worked on packaging design and final artwork for print

Education:
- Bachelor of Design, School of Visual Arts, Delhi (2015–2018)

Skills:
- Tools: Adobe Photoshop, Illustrator, InDesign, Figma, Adobe XD
- Focus Areas: Branding, Typography, UI/UX, Print Design
- Strengths: Creativity, time management, team collaboration
- Familiar with Windows and MacOS environments
"""
predicted_category=job_recommendation(resume_file)
print("Predicted Category:",predicted_category)

Predicted Category: User Interface Designer


In [23]:
resume_file = """
Results-oriented financial analyst with 3+ years of experience in investment banking and financial modeling. 
Skilled in analyzing balance sheets, income statements, and forecasting financial trends. Proficient in using Excel, Power BI, and SQL to perform deep data analysis. 

Collaborated with cross-functional teams to prepare due diligence reports and valuation models for M&A deals. 
Supported senior bankers in client presentations, financial benchmarking, and preparing pitch books.

Key competencies:
- Risk assessment and compliance analysis
- Financial forecasting and budgeting
- Credit analysis and loan portfolio review
- Capital markets and securities research

Education:
- B.Com (Hons) in Finance, Delhi University
- Certified Financial Analyst (CFA) Level II candidate

Tools: MS Excel, Bloomberg Terminal, Python (pandas), Tableau

Looking forward to contributing to dynamic banking and finance environments through data-driven decision making and strategic insights.

"""
predicted_category=job_recommendation(resume_file)
print("Predicted Category:",predicted_category)

Predicted Category: Business Intelligence Analyst


In [24]:
import pickle
pickle.dump(rf_classifier,open('models/rf_classifier_job_recommendation.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('models/tfidf_vectorizer_job_recommendation.pkl','wb'))